In [1]:
import pandas as pd
import numpy as np
from pymongo import MongoClient, UpdateOne
from datetime import datetime, timedelta
import glob, os, shutil
import time

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By

### MongoDB Conexion

In [3]:
conn = MongoClient("localhost:27017")
db = conn.get_database("expenses_control")

In [4]:
move = db.get_collection('movements')

In [5]:
def remove_docs():
    return move.remove({})

### Downloading and preparing bank data

In [6]:
USERNAME = "jpello6"
PASSWORD = "fapell6"

In [7]:
def data_extraction():
    
    os.environ['WDM_LOG_LEVEL'] = '0'
    browser = webdriver.Chrome(ChromeDriverManager().install())
    wait = WebDriverWait(browser, 10)
    browser.maximize_window()
    browser.get("https://bancaonline.bankinter.com/gestion/login.xhtml")
    browser.implicitly_wait(5)
    browser.find_element_by_xpath("//a[contains(text(),'aceptar')]").click()
 
    username = browser.find_element_by_name("uid")
    password = browser.find_element_by_name("password")
    
    username.send_keys(USERNAME)
    password.send_keys(PASSWORD)
    browser.find_element_by_id("btEntrar").click()
    browser.implicitly_wait(5)
    
    charges = wait.until(ec.element_to_be_clickable((By.XPATH, '/html/body/main/div/div/section/div[1]/form[1]/section/div/div[7]/div[1]/div[2]/div[3]/ul/li[1]')))
    charges.click()
    
    download_charges_excel = wait.until(ec.element_to_be_clickable((By.XPATH, '/html/body/main/section/form/div[1]/span[4]/div/div/ul/li[1]/a')))
    download_charges_excel.click()
    
    return_home = wait.until(ec.element_to_be_clickable((By.XPATH, '//*[@id="menuCabecera"]/li[1]/a')))
    return_home.click()
    
    incomes = wait.until(ec.element_to_be_clickable((By.XPATH, '/html/body/main/div/div/section/div[1]/form[1]/section/div/div[1]/div[1]/div[2]/div[3]/ul/li[1]')))
    incomes.click()
    
    download_incomes_excel = wait.until(ec.element_to_be_clickable((By.XPATH, '/html/body/main/section/div[1]/span[1]/div[1]/form/ul/li[1]/a')))
    download_incomes_excel.click()

    
    time.sleep(5)

    browser.close()

In [8]:
def move_file():
    incomes_file = 'C:\\Users\\juanp\\Downloads\\movimientos.xls'
    charges_file = 'C:\\Users\\juanp\\Downloads\\Movimientos (1).xls'
    destination = 'C:\\Users\\juanp\\proyectos_varios\\expenses_control\\data'
    
    try:
        shutil.move(charges_file,destination)
        shutil.move(incomes_file, destination)
    except: 
        pass
    try:
        os.replace(charges_file, destination)
        os.replace(incomes_file, destination)
    except:
        pass

In [9]:
def insert_object(df, collection):
    '''
    This function inserts all the information from the dataframe we created with the previous function as a Mongodb object.
    This object will have two field: title (name of the film or show) and reviews (array with all the reviews users have made)
    '''

    collection.insert_many(df.to_dict('records'))

In [10]:
def data_insert(path_charges, path_incomes, collection):
    
    incomes = pd.read_excel(path_incomes)[3:]
    incomes.drop(['Unnamed: 1', 'Unnamed: 4'], axis = 1, inplace=True)
    incomes.rename(columns={'IBAN: ES6701280067740100026987':'date', 'Unnamed: 2':'description', 
                            'Unnamed: 3':'quantity'}, inplace=True)
    
    charges = pd.read_excel(path_charges)
    list_index = charges.index[charges["Número de tarjeta: Visa Clasi (....2133)"]=='Total Crédito'].tolist()
    for number in list_index:
        limit_index = number
    charges = charges[5:limit_index]
    charges.drop(['Unnamed: 2'], axis = 1, inplace=True)
    charges.rename(columns={'Número de tarjeta: Visa Clasi (....2133)':'date', 'Unnamed: 1':'description', 
                            'Unnamed: 3':'quantity'}, inplace=True)
    charges['date'] = charges['date'].apply(lambda x: x.strftime('%d/%m/%Y'))
    
    complete = pd.concat([incomes, charges])
    
    current_month = datetime.today().date().replace(day=1).strftime('%m/%Y')
    first_day_prev_month = datetime.today().replace(day=1) - timedelta(days=1)
    prev_month = first_day_prev_month.strftime('%m/%Y')
    
    old = pd.DataFrame(list(move.find({'date':{'$regex':prev_month}},{'_id':0})))

    option = input('What action do you want to take? - update or insert').lower()
    
    if option != 'update' or option != 'insert':
        print('Invalid action. Action must be either update or insert.')
        option = input('What action do you want to take? - update or insert').lower()
        
        if option == 'insert':
            insert_object(complete, move)
            return 'Data succesfully inserted'
    
        elif option == 'update':
            complete_up = complete.loc[complete['date'].str.contains(current_month)]
            final_df = pd.concat([old, complete_up])
            final_df.drop_duplicates(keep = False, inplace = True)
        
            if len(final_df) == 0:
                return 'Nothing to update'
            else:
                insert_object(final_df, move)
                return 'Data succesfully updated'
        
    
    
        

In [11]:
def my_bank_movements():
    
    try:
    
        data_extraction()
        
    except:
        
        return 'There has been a problem with the conexion. Please, try again later'
    
    move_file()

    return data_insert('data\\movimientos.xls', 'data\\Movimientos (1).xls', move)
    

In [60]:
cash_out = ['PAGO BIZUM A','TRANS /Juan Maria Perez Lloren','CAJERO BANKINTER']
transport = ['Wib Advance','ALSA','METRO DE','EasyPark','Cabify','Uber','RENFE','BLABLACAR','Busbud','SANTA ANA,S.L',
             'CARMEN PARDO VALCARC','ORIENTE','FLOWBIKES']
shopping = ['MOONDAY','THEBRUBAKER','SUPERMERCADO','UNIDE','MARKET','SUPERCOR','DECATHLON','EXPRESS','CORTEFIEL','POLLOFRE',
           'ALBIE','FERRETERIA','PELUQUEROS']
restaurants = ['PEREGRINO','KUNFU CASTILLA','LA LLAMA','DIVINA DOLORES','TONELADS','RESTAURACION MARTI','CAFE'
               ,'NEW YORK BURGER','MARTINGALA','RESTAURANTE','HELADERIA','EL CHILITO',
              'TOPPINGS AND SALADS','BEFFCIOUS','TASTY POKE','SOTTGODIS','CORREQUESEQUEMA','BISTROTECA','ESPUMOSA'
               ,'SUPER ABUELA','SHUMARA','BAR DA PRAIA','HIPODROMO','EL PATIO MAHOU']
other_activities = ['CINESA','SINTRA']
subscriptions = ['HBO','MEDIUM', 'CLUB DEPORTIVO','MICAPPITAL']
accomodation = ['LATAS SURF','HOSTELWORLD']


In [44]:
lista = [cash_out, transport, shopping, restaurants, other_activities, subscriptions, accomodation]

In [61]:
def categorization(value):
    for cash in cash_out:
        if cash in value:
            return 'cash-out'
        else:
            pass
    for vehicle in transport: 
        if vehicle in value:
            return 'transport'
        else:
            pass
    for place in restaurants:
        if place in value:
            return 'restaurant'
        else:
            pass
    for activity in other_activities:
        if activity in value:
            return 'other activity'
        else:
            pass
    for subscription in subscriptions:
        if subscription in value:
            return 'subscription'
        else:
            pass
    for stay in accomodation:
        if stay in value:
            return 'accomodation'
        else:
            pass

In [62]:
test['cat_try'] = list(map(categorization,test.description))

In [63]:
test[test.cat_try == 'transport']

,date,description,quantity,cat_try
41,29/06/2021,Wib Advance Mobility S,-4.59,transport
48,05/07/2021,"METRO DE MADRID, S.A. 1",-20.00,transport
57,10/07/2021,EasyPark Espana S.L.U.,-2.30,transport
61,11/07/2021,Cabify Espana,-12.95,transport
65,17/07/2021,Wib Advance Mobility S,-4.43,transport
69,18/07/2021,EasyPark Espana S.L.U.,-0.88,transport
78,28/07/2021,Wib Advance Mobility S,-4.51,transport
109,02/08/2021,RENFE VIRTUAL INTERNET,-28.45,transport
110,02/08/2021,EasyPark Espana S.L.U.,-0.73,transport
112,04/08/2021,ALSA INTERNET,-24.00,transport


In [58]:
test

,date,description,quantity,cat_try
0,02/07/2021,RECIBO VISA CLASICA,-509.52,None
1,05/07/2021,PAGO BIZUM DE GONZALO;SUARDIAZ,4.00,None
2,05/07/2021,PAGO BIZUM DE Fernando;Munoz;T,4.00,None
3,05/07/2021,PAGO BIZUM A Maria;Miguel;Narc,-13.00,cash-out
4,05/07/2021,PAGO BIZUM DE ELENA;LLORENTE;P,146.00,None
...,...,...,...,...
257,21/09/2021,DZACARD.COM,-42.37,None
258,23/09/2021,CAJERO BANKINTER OF.0067,-40.00,cash-out
259,24/09/2021,LA ESPUMOSA,-67.00,restaurant
260,28/09/2021,CAJERO BANKINTER OF.0067,-70.00,cash-out


In [56]:
list(test.cat_try)

[None,
 None,
 None,
 'cash-out',
 None,
 'cash-out',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'cash-out',
 'cash-out',
 None,
 None,
 None,
 'cash-out',
 'cash-out',
 'cash-out',
 'cash-out',
 None,
 'cash-out',
 'cash-out',
 'cash-out',
 'cash-out',
 'cash-out',
 'cash-out',
 None,
 'cash-out',
 'cash-out',
 None,
 'cash-out',
 None,
 'cash-out',
 'cash-out',
 'cash-out',
 'cash-out',
 'transport',
 None,
 None,
 None,
 None,
 None,
 'restaurant',
 'transport',
 None,
 None,
 None,
 'restaurant',
 'restaurant',
 'restaurant',
 'restaurant',
 None,
 'transport',
 'subscription',
 None,
 'accomodation',
 'transport',
 None,
 None,
 None,
 'transport',
 None,
 'accomodation',
 'restaurant',
 'transport',
 'restaurant',
 'accomodation',
 'accomodation',
 'accomodation',
 'restaurant',
 'subscription',
 None,
 None,
 'transport',
 None,
 'cash-out',
 'cash-out',
 'cash-out',
 None,
 'cash-out',
 'cash-out',
 'cash-out',
 None,
 None,
 None,
 None,
 None,
 None,
 Non

In [22]:
test = pd.DataFrame(list(move.find({},{'_id':0})))

In [24]:
t = test.drop_duplicates(keep='first')